# Исследование надежности заемщиков


## Загрузка и изучение данных

In [60]:
import os

import pandas as pd

In [61]:
BASE_DIR = os.getcwd()
data = pd.read_csv(f'{BASE_DIR}/datasets/data.csv')

In [62]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [64]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


Всего в таблице 12 столбцов, тип данных столбцов различается.  
  
`children` — количество детей в семье  
`days_employed` — общий трудовой стаж в днях  
`dob_years` — возраст клиента в годах  
`education` — уровень образования клиента  
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose` — цель получения кредита  

### **Выводы**

  
Каждая строка таблицы содержит информацию об определенном физическом лице, запрашивающем кредитные средства на известную цель. Из таблицы можем видеть семейное положение, количество детей, возврат, трудовой стаж.   
В столбце `children` значения от -1 до 20.  
В столбце `days_employed` есть отрицательные значения, также среднее по столбцу - `63046.497661` дня, что необходимо изучить подробнее.  Минимальное значение приззнака `dob_years` - 0, что также необходимо исправить.

TO DO:  
- проверить пропуски и дубликаты   
- проверить столбец `children` на наличие артефактов и некорректных значений  
- проверить наличие минусовых значений в столбце `days_employed` и выявить причину данного явления  
- исправить значение возраста 0  
- привести к единому регистру столбец `education` и `family_status`  
- downcast типов данных  

## Предобработка данных

### Обработка пропусков

In [65]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. `days_employed` и `total_income`. К пропускам в столбце `days_employed` вернёмся позже, сначала обработаем пропуски первом столбце. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можно медианным значением по каждому типу из столбца `income_type`.

In [66]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [67]:
data[data['days_employed'] < 0]['days_employed'].count()

15906

В столбце days_employed множество отрицательных значений, что противоречит сути данных содержащихся в столбце. Заменим их на положительные.

In [68]:
data['days_employed'] = data['days_employed'].abs()

Изучим значения трудового стажа `days_employed` по типам занятости.

In [69]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) аномально большие значения. Нет очевидного подходя к исправлению таких значений, поэтому оставим их как есть.

### Обработка выбросов в столбце `children`

In [70]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение -1 явно ошибочно, а 20 слишком маловероятно, доля аномальных значений невелика, просто удалим эти объекты.

In [71]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [72]:
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Проверим результат

### Удаление пропусков

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`

In [73]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Повторно проверим датасет на наличие пропусков

In [74]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

В столбце `total_income` заменим `float` на `int`

In [75]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [76]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В столбце `education` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.

In [77]:
data['education'] = data['education'].str.lower()

Посмотрим наличие полных дубликатов

In [78]:
data.duplicated().sum()

71

Удалим их

In [79]:
data = data.drop_duplicates()

### Категоризация данных

Перед нами поставлены следующая задача:

Ответить на вопрос влияет ли семейное положение и количество детей клиента на факт погашения кредита.

Разобьем данные на категории следующим образом: 

- по доходу (`total_income`):  
    - `E` - < 30.000;
    - `D` - от 30.001 до 50.000;
    - `C` - от 50.001 до 200.000;
    - `B` - от 200.001 до 1.000.000
    - `A` - > 1.000.000    
   
    
      
- по цели взятия кредита (необходимо укрупнить существующие категории);

Начнём с первого пункта, назначим карегории в зависимости от доходов.  
Используем собственную функцию `categorize_income()` и метод `apply()`.

In [80]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [81]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Укрупним категории по целям взятия кредитов

In [82]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все цели можно разделить на 4 группы:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Напишем функцию с `categorize_purpose()` и метод `apply()` добавим признак с категорией в датасет.

In [83]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [84]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Ответим на ряд вопросов о данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [85]:
data.groupby('children')['debt'].agg(['count','mean'])

,count,mean
children,,
0,14091,0.075438
1,4808,0.092346
2,2052,0.094542
3,330,0.081818
4,41,0.097561
5,9,0.000000


In [86]:
data['parent'] = data['children'] > 0
data.groupby('parent')['debt'].agg(['count','mean'])

,count,mean
parent,,
False,14091,0.075438
True,7240,0.092403


Кредиторы с детьми чаще имеют долги по кредитам, чем кредиторы без детей. Причём нет явной зависимости  от количества детей, скорее от их наличия или отсутствия.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [87]:
data.groupby('family_status')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
family_status,,
вдовец / вдова,951,0.066246
в разводе,1189,0.070648
женат / замужем,12261,0.075606
гражданский брак,4134,0.093130
Не женат / не замужем,2796,0.097639


Между семейным положением и возвратом кредита есть зависимость.
Вдовцы и вдовы имеют меньший процент задолженностей, не женатые и не замужние самый высокий.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [88]:
data.groupby('total_income_category')['debt'].agg(['count','mean'])

,count,mean
total_income_category,,
A,25,0.080000
B,5014,0.070602
C,15921,0.084982
D,349,0.060172
E,22,0.090909


Число кредиторов сильно разнится в различных группах. В категориях с самым низким и с самым высоким доходов данных не достаточно для того чтобы сделать какой-либо вывод, то же можно сказать и о категории D. Если сравнивать категории B и C (2 и 3 место по доходам), то есть корреляция между доходом и возвратом кредита.

#### Как разные цели кредита влияют на его возврат в срок?

In [89]:
data.groupby('purpose_category')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
purpose_category,,
операции с недвижимостью,10751,0.072551
проведение свадьбы,2313,0.079118
получение образования,3988,0.092528
операции с автомобилем,4279,0.093480


Процент возврата кредитов отличается в зависимости от целей кредита. Наиболее добросовестными оказались кредиторы по недвижимости. Наибольший процент должников по кредитам на автомобили.

### Вывод

В результате исследования платёжеспособности заёмщиков кредитов были выявлены следующие зависимости:

- Из рассмотренных показателей однозначной зависимости не выявлено по критерию заработной платы. Требуется или получить больше данных по кредиторам с максимальными и минимальными заработками или переопределить границы разбивки на категории.  
  
- На выплату по кредиту влияет наличие или отсутствие детей, но не их количество.  
  
- Успешная выплата кредита так же зависит от целей кредита и семейного положения кредитора.  
  
**Попробуем нарисовать портет идеального заёмщика**:

Идеальный заёмщик - вдовец без детей с доходом выше среднего, берущий кредит на недвижимость.